# Wordle

In [40]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()


client =AzureOpenAI(
    api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["OPENAI_ENDPOINT"],
    api_key=os.environ["OPENAI_API_KEY"],
    azure_deployment=os.environ["OPENAI_DEPLOYMENT"],
)

response = client.chat.completions.create(
    model="gpt-4.1",  # Azure 版本可改为你的部署名称
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about large language models."}
    ]
)

print(response.choices[0].message.content)

Vast seas of data,  
Words weaving thought into code—  
Silent mind unfolds.


Load the model tokenizer from Hugging face:

In [31]:
from transformers import AutoTokenizer

base_model_id = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

## Setting up prompts to play Wordle

In [41]:
SYSTEM_PROMPT = """
You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close 
your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to 
first add your step by step thought process within <think> </think> 
tags. Then, return your guessed word in the following format: 
<guess> guessed-word </guess>.
"""


In [42]:
from dataclasses import dataclass
from enum import Enum
from typing import List


class LetterFeedback(Enum):
    CORRECT = "✓"
    WRONG_POS = "-"
    WRONG_LETTER = "x"


@dataclass
class GuessWithFeedback:
    guess: str
    feedback: List[LetterFeedback]

    def __repr__(self) -> str:
        """Returns a readable string showing the guess alongside
        its letter-by-letter feedback."""
        feedback_str = " ".join(f"{letter}({fb.value})" for letter, fb in zip(self.guess, self.feedback))
        return f"{self.guess} → Feedback: {feedback_str}"

In [43]:
def render_user_prompt(past_guesses: List[GuessWithFeedback]) -> str:
    """Creates a user-facing prompt that includes past guesses 
    and their feedback."""
    prompt = "Make a new 5-letter word guess."
    if past_guesses:
        prompt += "\n\nHere is some previous feedback:"
        for i, guess in enumerate(past_guesses):
            prompt += f"\nGuess {i+1}: {guess}"
    return prompt

In [44]:
def render_prompt(past_guesses: List[GuessWithFeedback]):
    """Formats a full chat prompt using a system message, user 
    prompt, and assistant preamble to start the model's 
    step-by-step reasoning."""
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": render_user_prompt(past_guesses)
        },
        {
            "role": "assistant",
            "content": "Let me solve this step by step.\n<think>"
        }
    ]

    return messages

In [45]:
def generate_completion(prompt: str) -> str:

    response = client.chat.completions.create(
        model="gpt-4.1",  # Azure 版本可改为你的部署名称
        messages=prompt
    )
    print(response.choices[0].message.content)

    return response.choices[0].message.content

## Play a single turn of Wordle

Start by setting up the prompt with feedback for two prior guesses:

In [46]:
secret_word = "CRAFT"

past_guesses = [
    GuessWithFeedback(
        "CRANE", [
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.WRONG_LETTER, 
            LetterFeedback.WRONG_LETTER,
        ]),
    GuessWithFeedback(
        "CRASH", [
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.WRONG_LETTER, 
            LetterFeedback.WRONG_LETTER,
        ]),
]

prompt = render_prompt(past_guesses)

print(prompt)

[{'role': 'system', 'content': '\nYou are playing Wordle, a word-guessing game.\n\n### Game Rules:\n- You have **6 tries** to guess a secret **5-letter** word.\n- Each guess must be a valid **5-letter English word**.\n- After each guess, you will receive feedback indicating how close \nyour guess was.\n\n### Feedback Format:\nEach letter in your guess will receive one of three symbols:\n1. ✓ : The letter is in the word and in the CORRECT position.\n2. - : The letter is in the word but in the WRONG position.\n3. x : The letter is NOT in the word.\n\n### Example:\nSecret Word: BRISK\n\nGuess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)\nGuess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)\nGuess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)\n\n### Response Format:\nThink through the problem and feedback step by step. Make sure to \nfirst add your step by step thought process within <think> </think> \ntags. Then, return your guessed word in the following format: \n<guess> guessed-word </g

Prompt the base model and examine its response:

In [47]:
base_completion = generate_completion(prompt)

<think>
Let’s review the clues:

From Guess 1: CRANE → C(✓) R(✓) A(✓) N(x) E(x)
- C, R, and A are correct and in the right positions: C at 1, R at 2, A at 3.
- N and E are not in the word at all.

From Guess 2: CRASH → C(✓) R(✓) A(✓) S(x) H(x)
- C, R, A are again confirmed correct spots.
- S and H are not in the word at all.

Summary:
- The word starts with CRA.
- The last two letters are NOT N, E, S, H.

So possibilities are words that start with CRA and end with two letters not including N, E, S, H.

Possible words that fit: CRAMP, CRACK, CRAGS, CRAVE, CRAZY, CRABY, CRAFT, CRASH (but S/H were eliminated), etc.

- CRAVE has an E (bad), crazy has Z and Y (allowed so far), crack (K is allowed), cramp (M and P are fine), craft (T is allowed), CRAGS (S not allowed), CRABY (B Y are ok).

Let’s try CRAMP, as it fits all criteria and both letters (M, P) have not been ruled out.
</think>
<guess> CRAMP </guess>


Now prompt the RFT model and see how it responds:

## Playing a game of wordle

Start by setting up a helper function that gets feedback on a guess:

In [48]:
import re

def get_feedback(guess: str, secret_word: str) -> List[LetterFeedback]:
    valid_letters = set(secret_word)
    feedback = []
    for letter, secret_letter in zip(guess, secret_word):
        if letter == secret_letter:
            feedback.append(LetterFeedback.CORRECT)
        elif letter in valid_letters:
            feedback.append(LetterFeedback.WRONG_POS)
        else:
            feedback.append(LetterFeedback.WRONG_LETTER)
    return feedback

Now create a `next_turn` function that incorporates feedback on a guess into the prompt to the LLM:

In [49]:
def next_turn(
    past_guesses: List[GuessWithFeedback], 
    secret_word: str
):
    prompt = render_prompt(past_guesses)
    completion = generate_completion(prompt)
    match = re.search(
        r"<guess>\s*(.*?)\s*</guess>", completion, re.DOTALL
    )
    if not match:
        raise RuntimeError("invalid guess")
    
    guess = match.group(1).upper()
    feedback = get_feedback(guess, secret_word)
    past_guesses.append(GuessWithFeedback(guess, feedback))
    print("\n\n")
    print(("-" * 100) + "\n")
    for past_guess in past_guesses:
        print(past_guess)
    
    if guess == secret_word:
        print("🎉 SUCCESS 🎉")
    elif len(past_guesses) >= 6:
        print("❌ better luck next time... ❌")

Try playing with the base model:

In [51]:
secret_word = "BRICK"
past_guesses = []

next_turn(past_guesses, secret_word)

<think>
Since this is the first guess and I don't have any feedback yet, I'll pick a common starter word that covers frequently used letters and a mix of vowels and consonants. Words like "CRANE," "SLATE," or "AUDIO" are often good for understanding the distribution of letters. For broad coverage, I'll go with "SLATE."
</think>
<guess> SLATE </guess>



----------------------------------------------------------------------------------------------------

SLATE → Feedback: S(x) L(x) A(x) T(x) E(x)


In [52]:
next_turn(past_guesses, secret_word)

<think>
The feedback for the guess "SLATE" tells me that none of its letters (S, L, A, T, E) are present in the secret word at all.
This eliminates those five letters from being in any future guesses.
I need to guess a 5-letter word that contains none of the following letters: S, L, A, T, E.

I'll brainstorm 5-letter words using only the remaining letters. Some possible options might be words like "ROUND," "BUMPY," "PRONG," or "WHIRL". Let me check each word to ensure it has no forbidden letters.

"ROUND":
R (okay)
O (okay)
U (okay)
N (okay)
D (okay)

None of S, L, A, T, E appear in "ROUND".
</think>
<guess> ROUND </guess>



----------------------------------------------------------------------------------------------------

SLATE → Feedback: S(x) L(x) A(x) T(x) E(x)
ROUND → Feedback: R(-) O(x) U(x) N(x) D(x)


In [53]:
next_turn(past_guesses, secret_word)

<think>
Let's analyze the feedback:

Guess 1: SLATE → S(x) L(x) A(x) T(x) E(x)
- None of the letters S, L, A, T, E are in the word.

Guess 2: ROUND → R(-) O(x) U(x) N(x) D(x)
- R is in the word but not in the second position.
- O, U, N, D are not in the word.

From this, the correct word must:
- Include R, but not in the second position.
- Not contain S, L, A, T, E, O, U, N, or D.

Let's find possible positions for R: 1st, 3rd, 4th, or 5th.
Let's think of a word that follows these conditions.

Possible letters: R (not in 2nd), and other unused letters such as I, C, P, M, Y, H, G, B, F, Q, V, W, X, Z (which are not yet ruled out).

Trying to construct a common word. For example: 
- "PRICK": P, R, I, C, K. R in position 2—NOT allowed.
- "CRISP": C, R, I, S, P. S not allowed.

Try R in position 3: "MIRTH" (M, I, R, T, H). T not allowed.

R in position 4: "WHIRL" (W, H, I, R, L). L not allowed.

R in position 5: "FIRER" (F, I, R, E, R). E not allowed.

Try "BIRCH":
- B, I, R, C, H. 
- R in

In [54]:
next_turn(past_guesses, secret_word)

<think>
Let's analyze each guess and the feedback:

Guess 1: SLATE → S(x) L(x) A(x) T(x) E(x)
- S, L, A, T, E are NOT in the word.

Guess 2: ROUND → R(-) O(x) U(x) N(x) D(x)
- R is in the word but NOT in position 1.
- O, U, N, D are NOT in the word.

Guess 3: BIRCH → B(✓) I(-) R(-) C(✓) H(x)
- B is CORRECT at position 1.
- C is CORRECT at position 4.
- I and R are both in the word, but NOT in positions 2 or 3 for I, or in positions 2 or 3 for R.
- H is NOT in the word.

Summary:
- B (position 1), C (position 4)
- I somewhere (NOT 2 or 3)
- R somewhere (NOT 2 or 3, and not 1 since that's B, and not 4 since that's C, leaving only position 5)
- Letters NOT allowed: S, L, A, T, E, O, U, N, D, H
- Letter placement:
  - 1: B
  - 2: (not I or R)
  - 3: (not I or R)
  - 4: C
  - 5: likely R (the only spot for R left, since it's not 1 or 3 or 2 or 4)

I must place I in either position 2 or 5, but position 5 is likely R, as explained above. However, I cannot put I at 2 or 3 based on "BIRCH" feed